In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Equity Option - Pricing and risk

Attributes
----------
equity option
inline valuations
market value
exposure
option delta
black scholes
"""

toggle_code("Toggle Docstring")

In [2]:
# %load_ext lab_black
# %load_ext nb_black

# Equity Options - Pricing and risk using LUSID's native valuation engine

In this notebook, we  demonstrate how you can generate a price and risk for equity options using LUSID's valuation engine. We use the Black-Scholes model to produce a price. We also caclulate the option delta, where delta is defined as the rate of change of the price of the option versus the price of the underlying equity instrument. 

For more details on modelling `EquityOptions` and booking associated cash flows, see [Knowlege Base article](https://support.lusid.com/knowledgebase/article/KA-01755/en-us).


## Table of Contents:
* [1. Create instruments](#1-create-instruments)
* [2. Market data](#2-market-data)
* [3. Run valuations](#3-run-valuations)


In [3]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken
from copy import deepcopy


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.9476.0


In [4]:
# LUSID Variable Definitions
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)
complex_market_data_api = api_factory.build(lu.api.ComplexMarketDataApi)

In [5]:
# Define scopes
scope = "BlackScholesValuations"
quotes_scope = "BlackScholesValuations"
portfolio_code = "equityOptionPortfolio"

## 1. Create Instruments <a id='1-create-instruments'></a>

First we need to create two instruments:

1. An Amazon equity, modelled as `Equity` in LUSID.
2. An option on this equity, modelled as `EquityOption` in LUSID.

### 1.1 Create the underlying instrument

In order to create an Option on an `Equity` Instrument, we must first create that underlying `Equity` instrument.

In [6]:
# upload AMZN equity

equity = lm.Equity(dom_ccy="USD", instrument_type="Equity",)

equity_definition = lm.InstrumentDefinition(
    name="Amazon.com",
    identifiers={
        "ClientInternal": lm.InstrumentIdValue("AMZN"),
        "RIC": lm.InstrumentIdValue("AMZN"),
    },
    definition=equity,
)

# upsert the instrument
upsert_request = {"AMZN": equity_definition}
upsert_response = instruments_api.upsert_instruments(request_body=upsert_request, scope=scope)
equity_luid = upsert_response.values["AMZN"].lusid_instrument_id
print(equity_luid)

LUID_00003DUC


### 1.2 Create the Equity Option

Now that we have created the underlying `Equity`, we can create an `EquityOption` on that instrument. This option is a "European" option meaning that it can only be exercised on the expiry date.

In [7]:
def option_instrument_definition(
    option_name,
    option_identifier,
    start_date,
    option_maturity_date,
    option_settle_date,
    delivery_type,
    option_type,
    strike,
    dom_ccy,
    underlying_code,
    underlying_identifier,
):

    option = lm.EquityOption(
        instrument_type="EquityOption",
        start_date=start_date,
        option_maturity_date=option_maturity_date,
        option_settlement_date=option_settle_date,
        delivery_type=delivery_type,
        option_type=option_type,
        strike=strike,
        dom_ccy=dom_ccy,
        underlying_identifier=underlying_identifier,
        code=underlying_code,
    )

    return option


def create_option_instrument(option):

    # define the instrument to be upserted
    option_definition = lm.InstrumentDefinition(
        name=option_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(option_identifier)},
        definition=option,
    )

    # upsert the instrument
    upsert_request = {option_identifier: option_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request, scope=scope)
    option_luid = upsert_response.values[option_identifier].lusid_instrument_id
    print(option_luid)

Use the function above to create an `EquityOption`. See the API documentation for the available options for each attribute.
Filter for instrumentType=`EquityOptions` under [UpsertInstruments](https://www.lusid.com/docs/api/#operation/UpsertInstruments).

In [8]:
option_identifier = "AMZN_31/12/22_C168"
option_name = option_identifier
start_date = datetime(2022, 1, 1, 0, tzinfo=pytz.utc)
option_maturity_date = datetime(2022, 12, 31, 0, tzinfo=pytz.utc)
option_settle_date = datetime(2022, 12, 31, 0, tzinfo=pytz.utc)
delivery_type = "Cash"
option_type = "Call" #see API docs for all configuration options
strike = 168
dom_ccy = "USD"
underlying_code = "AMZN"
underlying_identifier = "RIC"

option_instrument = option_instrument_definition(
    option_name,
    option_identifier,
    start_date,
    option_maturity_date,
    option_settle_date,
    delivery_type,
    option_type,
    strike,
    dom_ccy,
    underlying_code,
    underlying_identifier,
)


create_option_instrument(option_instrument)

LUID_00003DUD


## 2. Market data <a id='2-market-data'></a>

To value and risk the option, we need three sets of market data:

1. Equity prices for the underlying `Equity`
2. Votality surface data for the underlying stock
3. Discount curve

The underlying equity prices (#1 above) are loaded into the standard `Quotes` store (via [UpsertQuotes](https://www.lusid.com/docs/api/#operation/UpsertQuotes)) whereas the volatility surface (#2) and [discount curve](https://support.lusid.com/knowledgebase/article/KA-01715/en-us) (#3) are loaded in to the `ComplexMarketData` store. The discount curve is used to calculate the risk-free rate required by the Black-Scholes model.

### 2.1 Upload underlying equity prices

In [9]:
prices = pd.read_csv("data/equity_options_data.csv")
prices.head()

,date,price
0,03/01/2022,171.4000
1,04/01/2022,162.5200
2,05/01/2022,163.3500
3,06/01/2022,164.2500
4,07/01/2022,165.5500


In [10]:
ric = "AMZN"
currency = "USD"

# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=ric,
                instrument_id_type="RIC",
                quote_type="Price",
                field="mid",
            ),
            effective_at=to_date(row["date"]),
        ),
        metric_value=lm.MetricValue(value=row["price"], unit=currency),
    )
    for index, row in prices.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if len(response.failed) > 0:
    print(
        f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

Some failures occurred during quotes upsertion, 0 did not get loaded into LUSID.


### 2.2 Upload volatility surface for the underlying equity

For the purposes of demonstration we load a flat vol surface which sets the implied volatility to 60% for the Amazon equity. This is loaded against an ATM option which has a strike price equals to the spot price.

In [11]:
# Create new option where strike price equals the spot price for 1 March 2022

option_instrument_for_vol_surface = deepcopy(option_instrument)
option_instrument_for_vol_surface.strike = 156

In [12]:
# Set marketTypeDate=EquityVolSurfaceData on the API docs to see all parameter options
# https://www.lusid.com/docs/api/#operation/UpsertComplexMarketData

def upsert_eq_vol(scope, effective_at, market_asset, log_val):

    market_data_id = lm.ComplexMarketDataId(
        provider="Lusid",
        price_source="Lusid",
        effective_at=effective_at,
        market_asset=market_asset,
    )

    eq_vol_surface_data = lm.EquityVolSurfaceData(
        base_date=effective_at,
        instruments=[option_instrument_for_vol_surface],
        quotes=[lm.MarketQuote(quote_type="LogNormalVol", value=log_val)],
        market_data_type="EquityVolSurfaceData",
    )

    vol_request = complex_market_data_api.upsert_complex_market_data(
        scope=scope,
        request_body={
            market_asset: lm.UpsertComplexMarketDataRequest(
                market_data_id=market_data_id, market_data=eq_vol_surface_data
            )
        },
    )
    
    return vol_request.values

In [13]:
upsert_eq_vol(scope, "2022-03-01", "AMZN/USD/LN", 0.6)

{'AMZN/USD/LN': datetime.datetime(2022, 6, 20, 13, 28, 16, 263317, tzinfo=tzutc())}

### 2.2 Upload discount factors

We also upload [discount factors](https://support.lusid.com/knowledgebase/article/KA-01715/en-us). The Black-Scholes pricer requires discount factors to discount the expected future cashflow from the maturity of the option to the pricing date.

In [14]:
# scope used to store our market data
market_data_scope = scope
# the market data supplier
market_supplier = "Lusid"


def upsert_discount_factors(scope, effective_at, market_asset):

    # provide the structured data file source and it's document format
    complex_market_data = lm.DiscountFactorCurveData(
        base_date=datetime(2022, 3, 1, tzinfo=pytz.utc),
        dates=[
            datetime(2022, 3, 1, tzinfo=pytz.utc),
            datetime(2022, 5, 1, tzinfo=pytz.utc),
            datetime(2022, 7, 1, tzinfo=pytz.utc),
            datetime(2022, 9, 1, tzinfo=pytz.utc),
            datetime(2022, 11, 1, tzinfo=pytz.utc),
        ],
        discount_factors=[1.0, 0.9883, 0.9826, 0.9789, 0.9756,],
        market_data_type="DiscountFactorCurveData",
    )

    # create a unique identifier for our OIS yield curves
    complex_id = lm.ComplexMarketDataId(
        provider="Lusid",
        price_source="Lusid",
        effective_at=effective_at,
        market_asset=market_asset,
    )

    upsert_request = lm.UpsertComplexMarketDataRequest(
        market_data_id=complex_id, market_data=complex_market_data
    )

    # https://www.lusid.com/docs/api#operation/UpsertComplexMarketData
    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope, request_body={market_asset: upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload yield curve {response.failed}")

    print(f"{market_asset} yield curve uploaded into scope={scope}")


upsert_discount_factors(market_data_scope, "2022-03-01", "USD/USDOIS")

USD/USDOIS yield curve uploaded into scope=BlackScholesValuations


## 3. Valuations <a id='3-run-valuations'></a>

Finally we run valuations on the option to produce a price and risk metrics. We run the valuation using a custom recipe which has been configured to use the Black-Scholes model for options. There are three market data rules:


| Makret data key | Description |
| :--------------------- | :----------- |
| `Quote.RIC.*` | Load quotes for the underlying equity from the standard `Quotes` store  |
| `EquityVol.*.*.*` | Load volatility numbers from the `ComplexMarketDataStore` |
| `Rates.*.*` | Load discount curves from the `ComplexMarketDataStore` |



### 3.1 Create Valuation Recipe

In [15]:
# Set recipe code
recipe_code = "OptValuation"

# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
                        
            lm.MarketDataKeyRule(
                key="Quote.RIC.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            ),
            lm.MarketDataKeyRule(
                key="EquityVol.*.*.*",
                supplier="Lusid",
                data_scope=scope,
                price_source="Lusid",
                quote_type="Price",
                field="mid",
                quote_interval="30D.0D",
            ),
            lm.MarketDataKeyRule(
                key="Rates.*.*",
                supplier="Lusid",
                data_scope=scope,
                price_source="Lusid",
                quote_type="Price",
                field="mid",
                quote_interval="30D.0D",
            ),
        ],
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="BlackScholes",
                instrument_type="EquityOption",
                parameters="{}",
            )
        ],
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)


print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2022-06-20 13:28:17.631509+00:00.


### 3.2 Create inline valuation function

In [16]:
def run_inline_valuation(effective_at):

    recipe_id = lm.ResourceId(scope=scope, code=recipe_code)

    valuation_schedule = lm.ValuationSchedule(effective_at=effective_at)

    instruments = [
        lm.WeightedInstrument(
            quantity=100, holding_identifier="inst_001", instrument=option_instrument
        )
    ]

    metrics = [
        lm.AggregateSpec("Instrument/default/Name", "Value"),
        lm.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
        lm.AggregateSpec("Quotes/Price", "Value"),
        lm.AggregateSpec("Holding/default/Units", "Value"),
        lm.AggregateSpec("Valuation/PV/Amount", "Value"),
        lm.AggregateSpec("Valuation/Delta", "Value"),
        lm.AggregateSpec("Valuation/Risk/SpotDelta", "Value"),
        lm.AggregateSpec("Valuation/CleanPriceKey", "Value"),
        lm.AggregateSpec("Instrument/OTC/EquityOption/Strike", "Value"),
        lm.AggregateSpec("Instrument/OTC/EquityOption/OptionMaturityDate", "Value"),
        lm.AggregateSpec("Instrument/OTC/EquityOption/Code", "Value"),
    ]

    group_by = ["Instrument/default/Name"]

    valuation = aggregration_api.get_valuation_of_weighted_instruments(
        inline_valuation_request=lm.InlineValuationRequest(
            recipe_id=recipe_id,
            as_at=None,
            metrics=metrics,
            group_by=group_by,
            filters=None,
            sort=None,
            report_currency=None,
            equip_with_subtotals=None,
            valuation_schedule=valuation_schedule,
            instruments=instruments,
            local_vars_configuration=None,
        )
    )

    valuation_df = pd.DataFrame(valuation.data)

    rename_cols = {
        "Instrument/OTC/EquityOption/Code": "OptionCode",
        "Valuation/PV/Amount": "PresentValue",
        "Instrument/Definition/ContractSize": "ContractSize",
        "Instrument/OTC/EquityOption/Strike": "StrikePrice",
        "Instrument/OTC/EquityOption/OptionMaturityDate": "OptionMaturityDate",
        "Holding/default/Units": "Units",
        "Quotes/Price": "PriceOfUnderlying",
        "Valuation/CleanPriceKey": "OptionPrice",
        "Valuation/Delta": "Delta",
    }

    valuation_df = valuation_df.rename(columns=rename_cols)[rename_cols.values()]

    valuation_df = valuation_df.astype({"Units": "int32", "ContractSize": "int32"})

    valuation_df.OptionMaturityDate = valuation_df.OptionMaturityDate.apply(
        lambda x: x[:10]
    )

    return valuation_df

### 3.3 Run valuations

In this section, we run a valuation to produce the following:

* A present value for the option (using the Option's price which is calculated per Black-Scholes)
* A delta value for the option


The <b>Present Value</b> (also referred to as "Market Value") is calculated as follows:

> *Present Value = OptionPrice * ContractSize * Units*

In the example below, we have priced an "in the money" (ITM) option. It costs us \\$40.09 to buy the right (not not obligation) to purchase Amazon stock for \\$168.00 on 31 December 2022.

In [17]:
option_valuaton_df = run_inline_valuation("2022-03-01")
option_valuaton_df

,OptionCode,PresentValue,ContractSize,StrikePrice,OptionMaturityDate,Units,PriceOfUnderlying,OptionPrice,Delta
0,AMZN,"4,009.5771",1,168.0000,2022-12-31,100,171.4000,40.0958,0.6234
